In [1]:
import django 
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from sch.models import *
import datetime as dt
from django.db.models import CharField
from sch.actions import *
from sch.xviews import week

In [5]:
sch_all = Schedule.objects.all()
for sch in sch_all:
    print (sch)

S2022-5C
S2022-6C
S2022-7C
S2022-8C


In [7]:
wd = Workday.objects.get(slug='2022-11-01')
emptySlots = wd.slots.filter(employee=None).order_by('fillableByN')[0]._fillableBy()
sortByNeededHrs = employee.needed_hours(r)

<EmployeeManager [<Employee: Molly>, <Employee: Doug>, <Employee: Carrie>, <Employee: Lynlee>, <Employee: Andrew>]>

In [6]:
from sch.actions import *
from sch.xviews import week

In [52]:
empl_in_conflicting = list(slot.conflicting_slots.all().values_list('employee',flat=True).distinct())
empl_w_ptor = list(PtoRequest.objects.filter(workday=slot.workday.date).values_list('employee',flat=True).distinct())
empl_w_tdo  = list(TemplatedDayOff.objects.filter(sd_id=slot.workday.sd_id).values_list('employee',flat=True).distinct())
incompatible = list(set(empl_in_conflicting + empl_w_ptor + empl_w_tdo))
if None in incompatible:
    incompatible.remove(None)
incompatible_employees = Employee.objects.filter(id__in=incompatible)
fillableBy = Employee.objects.filter(shifts_available=slot.shift).exclude(id__in=incompatible)
print(fillableBy)


<EmployeeManager [<Employee: Brianna>, <Employee: Elisia>, <Employee: Jaden>, <Employee: Trisha>, <Employee: Tiffany>, <Employee: Nicki>]>


ZeroDivisionError: division by zero

In [17]:
emps = Employee.objects.in_other_slot(workday=Workday.objects.get(slug='2022-10-31'),shift=Shift.objects.get(name__contains='7C'))
emps = emps.filter(shifts_trained=Shift.objects.get(name__contains='7C'))
emps

<EmployeeManager []>

In [ ]:
shiftsPerWeek = 0
for sft in Shift.objects.filter(start__hour__gte=10, cls='CPhT').values_list('occur_days',flat=True):
    shiftsPerWeek += len(sft)
shiftsPerWeek

: 

## NIGHT PERCENTAGE OF TOTAL SHIFTS"

In [ ]:
NIGHT_PERCENTS = []
for emp in Employee.objects.all():
    if Slot.objects.filter(employee=emp).exists():
        perc = round(Slot.objects.filter(employee=emp,shift__start__hour__gte=10).count()/Slot.objects.filter(employee=emp).count()*100, 1)
        NIGHT_PERCENTS += [f"{emp}: {perc}% EVENINGS  {int(perc/4) *'+'}"]
        
NIGHT_PERCENTS
        

: 

## SLOTS FROM A WEEK

In [ ]:
def getTradableSlots(slot):
    print('INPUT: ', slot)
    emp = slot.employee
    sft = slot.shift
    wd = slot.workday
    weekdays = slot.workday.siblings_iweek
    daysOn = Slot.objects.filter(workday__in = weekdays,employee = emp).values('workday').distinct()
    daysOff = weekdays.exclude(pk__in=daysOn)
    print('daysOff:',list(daysOff.values_list('date__day',flat=True)))
    tenable = []
    slotDaysOff = Slot.objects.filter(workday__in=daysOff, shift__in=emp.shifts_available.all()).order_by('workday')
    print(slotDaysOff)
    for slotB in slotDaysOff:
        if emp in slot.fillableBy():
            if slotB.employee in slot.fillableBy():
                tenable.append(slotB)
    print('tenableTrades:',len(tenable),[f'{i.shift}>{i.workday.date.day}' for i in tenable])
    return tenable



: 

In [ ]:
slots = Slot.objects.filter(employee__name="Trisha",workday__iweek=43,workday__date__year=2022)
for slot in slots:
    print('-------------')
    print(slot.is_turnaround)
    getTradableSlots(slot)

: 

In [ ]:
wds = Workday.objects.filter(date__year=2022,ischedule=7)
ss = Slot.objects.filter(workday__in=wds)
ss

: 

In [ ]:
for wd in Workday.objects.filter(iweek=44):
    print(wd,"~~~Week:",wd.week)

: 

In [ ]:
n = Workday.objects.all().count()
print( f"TOTAL WORKDAYS: {n} " )

: 

In [ ]:
from django import forms
import datetime as dt
class ArticleForm(forms.Form):
     title = forms.CharField()
     pub_date = forms.DateField()

: 

In [ ]:
""" 
Janet’s ducks lay 16 eggs per day. She eats three for breakfast 
every morning and bakes muffins for her friends every day with 
four. She sells the remainder for $2 per egg. How much does she
make every day? SHOW YOUR CALCULATIONS
"""

# Define the number of eggs Janet's ducks lay per day.
eggs_per_day = 16

# Define the number of eggs Janet eats for breakfast every morning.
eggs_for_breakfast = 3

# Define the number of eggs Janet uses to bake muffins for her friends every day.
eggs_for_muffins = 4

# Define the price Janet sells her eggs for.
eggs_price = 2

# Calculate the number of eggs Janet has left over after eating breakfast and baking muffins.
eggs_leftover = eggs_per_day - eggs_for_breakfast - eggs_for_muffins

# Calculate the amount of money Janet makes selling her eggs.
eggs_money = eggs_leftover * eggs_price

# Print the number of eggs Janet has left over after eating breakfast and baking muffins.
print("Janet has " + str(eggs_leftover) + " eggs left over after eating breakfast and baking muffins.")

# Print the amount of money Janet makes selling her eggs.
print("Janet makes $" + str(eggs_money) + " selling her eggs.")

: 

In [ ]:
def turnarounds_withTrades ():
    tradeable = {}
    tas = Slot.objects.filter(is_turnaround=True)|Slot.objects.filter(is_preturnaround=True)
    slts = tas.filter(employee__evening_pref=True)
    empl_wantsThisSwapped = slts.filter(shift__start__hour__lt=10)
    for i in range(empl_wantsThisSwapped.count()):
        s = empl_wantsThisSwapped[i]
        if s.fillableBy().count() > 0:
            if not tradeable.get('pref'):
                tradeable['pref'] = []
            tradeable['pref'] += [s]
    empl_likesThisSlot = slts.filter(shift__start__hour__gte=10)
    for i in range(empl_likesThisSlot.count()):
        s = empl_likesThisSlot[i]
        if s.fillableBy().count() > 0:
            if not tradeable.get('nonpref'):
                tradeable['nonpref'] = []
            tradeable['nonpref'] += [s]
    return tradeable
    
x = turnarounds_withTrades()
x

: 

In [ ]:
t = {}
a = t.get('a')
print(a)

: 

In [ ]:
def weekSlots_for_Empl (yearN, weekN, employee):
    slots = Slot.objects.filter(workday__iweek=weekN,workday__date__year=yearN,employee=employee)
    for i in range(7):
        s = slots.filter(workday__iweekday=i)
        if s.exists() == False:
            yield None
        else:
            yield s.first()
            
def display_weekSlots_for_Empl (yearN, weekN, employee):
    slots = [slot for slot in weekSlots_for_Empl(yearN,weekN,employee)]
    wds = 'SUN MON TUE WED THU FRI SAT'.split(' ')
    empl = f'{employee}'
    
    print(f'{yearN}-Week_{weekN} employee schedule:')
    print(f'_________ {empl} __________')
    for i in range(7):
        print(f'{wds[i]}: {slots[i]}')
    print('==========================')

: 

In [ ]:
slots = weekSlots_for_Empl(2022,44,Employee.objects.get(name='Josh'))
slots = [slot for slot in slots]
display_weekSlots_for_Empl(2022,44,Employee.objects.get(name='Molly'))
display_weekSlots_for_Empl(2022,44,Employee.objects.get(name='Josh'))

: 

In [ ]:
def emp1_emp2_cowork_days (emp1,emp2,yearN,schN):
    jtsSlots = Slot.objects.filter(employee=emp1,workday__date__year=yearN,workday__ischedule=schN)
    nkiSlots = Slot.objects.filter(employee=emp2,workday__date__year=yearN,workday__ischedule=schN)
    dates = jtsSlots.union(nkiSlots)
    dates = dates.values('workday')
    for d in dates:
        if d in jtsSlots.values('workday'):
            if d in nkiSlots.values('workday'):
                yield d['workday']
                # "[207,217,223...]"
josh = Employee.objects.get(name="Josh")    
nicki = Employee.objects.get(name="Nicki")      
xs = emp1_emp2_cowork_days(josh,nicki,2022,5)
days = Workday.objects.filter(pk__in=xs)


print(f'COWORK DAYS FOR {josh} and {nicki}')
print("TOTAL FOR SCHEDULE:", days.count())
print(list(days.values_list('slug',flat=True)))



: 

In [ ]:
josh_slots_w_nicki = Slot.objects.filter(workday__in=days,employee=josh)
josh_slots_w_nicki 
for jslot in josh_slots_w_nicki:
    nslot = Slot.objects.get(workday=jslot.workday,employee=nicki)
    print(jslot,"  |  ",nslot)

: 

## Employee Training Comparisons

In [ ]:
vals = Employee.objects.annotate(n_shifts_trained=Count('shifts_trained')).order_by('n_shifts_trained').values_list('name','n_shifts_trained')
for val in vals:
    print ("%s is trained for %s shift(s)." % (val[0], val[1]))

: 

In [ ]:
Schedule.objects.create()

: 

In [ ]:
for emp in Employee.objects.all():
    print(f"{emp.name.ljust(20)} {emp.yrs_experience} yr. experience")

: 

In [ ]:
b = Employee.objects.get(name="David")
b.hire_date = dt.date(2022,6,1)
b.save()
b.hire_date

: 

# Low Option Employees

In [ ]:
loe = Employee.objects.filter(fte__gt=0).filter(n_trained__lt=4)
for l in loe:
    print (l.name, l.n_trained, l.shifts_trained.all())

: 

In [ ]:
import itertools as itr 

iters = list(itr.permutations(josh.shifts_available.all(), r=2))
iters

: 

: 